In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from datetime import datetime

In [2]:
seed = 42
cuda = True

batch_size = 64
log_interval = 1000

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

In [3]:
class Net1(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [4]:
class Lenet(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()
        self.conv2_n = 50
        self.hiddens = 50*16
        
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, self.conv2_n, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(self.hiddens, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), 2))
        x = F.tanh(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, self.hiddens)
        x = F.tanh(self.fc1(x))
        x = F.dropout(x, training=self.training, p=0.7)
        x = self.fc2(x)
        return F.log_softmax(x)

In [5]:
model = Lenet()
if cuda:
    model.cuda()

epochs = 40
lr = 0.01
momentum = 0.9

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [6]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
%%time

start = datetime.now()
new = start

for epoch in range(1, epochs + 1):
    train(epoch)
    test()
    old = new
    new = datetime.now()
    print(">> %s - %s" % (str(new-old), str(new-start)))

/home/yannick/bin/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.357072
Test set: Average loss: 0.0849, Accuracy: 9753/10000 (98%)
>> 0:00:08.834088 - 0:00:08.834088
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.088545
Test set: Average loss: 0.0644, Accuracy: 9809/10000 (98%)
>> 0:00:07.929191 - 0:00:16.763279
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.037827
Test set: Average loss: 0.0501, Accuracy: 9854/10000 (99%)
>> 0:00:07.408670 - 0:00:24.171949
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.067747
Test set: Average loss: 0.0449, Accuracy: 9875/10000 (99%)
>> 0:00:07.113462 - 0:00:31.285411
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.166929
Test set: Average loss: 0.0421, Accuracy: 9882/10000 (99%)
>> 0:00:07.299523 - 0:00:38.584934
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.044123
Test set: Average loss: 0.0382, Accuracy: 9897/10000 (99%)
>> 0:00:07.464540 - 0:00:46.049474
Train Epoch: 7 [0/60000 (0%)]	Loss: 0.025747
Test set: Average loss: 0.0404, Accuracy: 9890/10000 (99%)
>> 0:00:07.224323 - 0:00:53.273797
Train Epoch: 8 [0/60000 (0%

In [8]:
model

Lenet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=800, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)